In [1]:
!pip install numpy pandas matplotlib seaborn scikit-learn xgboost lightgbm catboost optuna shap eli5 tensorflow keras torch torchvision

In [2]:
import pandas as pd
import numpy as np

# Load the data
df_train = pd.read_csv("C:\Programming\Biotech_Dataset\Train2D.csv")
df_train.head()

,SMILES,Cls,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,C[C@H]1C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C(=...,0,NaN,NaN,1,10,126.291271,2.416707,4.779198,126.291271,...,11.160285,157.462679,1463.720435,7.318602,67876,151,492,548,40.194444,23.333333
1,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H...,0,NaN,NaN,1,10,122.306081,2.416770,4.780587,122.306081,...,11.127219,155.307010,1441.772471,7.067512,63926,148,478,531,41.194444,22.944444
2,C1=CC=C(C(=C1)C(=O)C2=CN=C(S2)NC3=CC=C(C=C3)N)Cl,0,NaN,NaN,0,0,28.571924,2.388244,4.711441,28.571924,...,9.874522,69.492153,329.038961,9.677616,1165,31,114,132,6.777778,4.833333
3,C1CCC(=O)C(C1)C2CC(=O)N(C2=O)C3=CC=C(C=C3)Br,0,NaN,NaN,0,0,27.097139,2.488091,4.835581,27.097139,...,10.022470,69.396660,349.031356,9.433280,902,34,112,134,7.138889,4.583333
4,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H...,0,NaN,NaN,1,10,132.321058,2.416813,4.810016,132.321058,...,11.267217,163.855774,1557.759855,7.382748,77098,162,524,585,43.618056,24.527778


In [3]:
df_train['Cls'].unique()

array([0, 1], dtype=int64)

In [4]:
df_train['Cls'].value_counts()

Cls
0    1720
1    1720
Name: count, dtype: int64

In [5]:
df_train['Cls'].isna().sum()

0

We've gotten a perfectly balanced dataset with 50% of the data being positive and 50% being negative. We can now use this dataset to train a model and evaluate its performance. But first, let's check out if there are any missing values in the dataset.

In [6]:
df_train.isna().sum()

SMILES         0
Cls            0
ABC         3440
ABCGG       3440
nAcid          0
            ... 
WPol           0
Zagreb1        0
Zagreb2        0
mZagreb1      81
mZagreb2       0
Length: 1615, dtype: int64

In [7]:
# let's fill all NaN values with 0
df_train.fillna(0, inplace=True)

In [8]:
df_train.describe()

,Cls,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
count,3440.000000,3440.0,3440.0,3440.000000,3440.000000,3440.000000,3440.000000,3440.000000,3440.000000,3440.000000,...,3440.000000,3440.000000,3440.000000,3440.000000,3.440000e+03,3440.000000,3440.000000,3440.000000,3440.000000,3440.000000
mean,0.500000,0.0,0.0,0.122965,0.475291,37.072754,2.367811,4.684533,37.072754,1.247193,...,10.174770,74.602004,424.650464,8.004186,1.779117e+08,45.921802,154.679651,179.994477,9.573169,6.629576
std,0.500073,0.0,0.0,0.366124,1.415025,18.780723,0.412210,0.813067,18.780723,0.217027,...,0.400537,18.841442,206.075359,1.191256,1.571118e+09,23.252123,71.016462,80.962620,5.843412,3.302986
min,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.921898,38.130322,136.100048,5.021635,1.260000e+02,9.000000,42.000000,44.000000,0.000000,2.333333
25%,0.000000,0.0,0.0,0.000000,0.000000,28.833567,2.394788,4.745224,28.833567,1.254065,...,9.966157,63.747217,328.122700,7.195197,1.277750e+03,34.000000,118.000000,137.750000,7.144097,5.111111
50%,0.500000,0.0,0.0,0.000000,0.000000,34.635766,2.436715,4.824731,34.635766,1.286094,...,10.185353,73.252532,383.113313,7.850231,2.025000e+03,42.000000,142.000000,166.000000,8.638889,5.944444
75%,1.000000,0.0,0.0,0.000000,0.000000,40.729804,2.479563,4.893096,40.729804,1.312111,...,10.388688,80.253821,452.095638,8.661321,3.348000e+03,51.000000,168.000000,198.000000,10.295139,7.000000
max,1.000000,0.0,0.0,3.000000,16.000000,165.270586,2.724670,5.449341,165.270586,1.381549,...,11.454226,190.869373,1955.002899,14.125169,4.270000e+10,204.000000,648.000000,724.000000,53.416667,30.416667


In [9]:
num_df_train = df_train.drop('SMILES', axis=1)
num_df_train.head()

,Cls,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0,0.0,0.0,1,10,126.291271,2.416707,4.779198,126.291271,1.226129,...,11.160285,157.462679,1463.720435,7.318602,67876,151,492,548,40.194444,23.333333
1,0,0.0,0.0,1,10,122.306081,2.416770,4.780587,122.306081,1.210951,...,11.127219,155.307010,1441.772471,7.067512,63926,148,478,531,41.194444,22.944444
2,0,0.0,0.0,0,0,28.571924,2.388244,4.711441,28.571924,1.298724,...,9.874522,69.492153,329.038961,9.677616,1165,31,114,132,6.777778,4.833333
3,0,0.0,0.0,0,0,27.097139,2.488091,4.835581,27.097139,1.290340,...,10.022470,69.396660,349.031356,9.433280,902,34,112,134,7.138889,4.583333
4,0,0.0,0.0,1,10,132.321058,2.416813,4.810016,132.321058,1.213955,...,11.267217,163.855774,1557.759855,7.382748,77098,162,524,585,43.618056,24.527778


In [10]:
num_df_train

,Cls,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0,0.0,0.0,1,10,126.291271,2.416707,4.779198,126.291271,1.226129,...,11.160285,157.462679,1463.720435,7.318602,67876,151,492,548,40.194444,23.333333
1,0,0.0,0.0,1,10,122.306081,2.416770,4.780587,122.306081,1.210951,...,11.127219,155.307010,1441.772471,7.067512,63926,148,478,531,41.194444,22.944444
2,0,0.0,0.0,0,0,28.571924,2.388244,4.711441,28.571924,1.298724,...,9.874522,69.492153,329.038961,9.677616,1165,31,114,132,6.777778,4.833333
3,0,0.0,0.0,0,0,27.097139,2.488091,4.835581,27.097139,1.290340,...,10.022470,69.396660,349.031356,9.433280,902,34,112,134,7.138889,4.583333
4,0,0.0,0.0,1,10,132.321058,2.416813,4.810016,132.321058,1.213955,...,11.267217,163.855774,1557.759855,7.382748,77098,162,524,585,43.618056,24.527778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3435,1,0.0,0.0,0,0,35.981349,2.454850,4.909699,35.981349,1.240736,...,10.330160,64.664490,414.197714,7.020300,2578,48,152,178,11.062500,6.250000
3436,1,0.0,0.0,0,0,38.070420,2.424294,4.848587,38.070420,1.312773,...,10.287218,64.627285,381.147727,7.940578,2372,42,150,175,7.638889,6.583333
3437,1,0.0,0.0,0,2,49.867009,2.452215,4.904315,49.867009,1.278641,...,10.597809,83.963738,538.266794,7.475928,5435,66,210,245,12.395833,8.277778
3438,1,0.0,0.0,1,0,47.111260,2.439942,4.879884,47.111260,1.273277,...,10.552709,73.833590,541.071988,9.837673,4596,63,194,229,13.305556,8.138889


In [11]:
df_train

,SMILES,Cls,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,C[C@H]1C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C(=...,0,0.0,0.0,1,10,126.291271,2.416707,4.779198,126.291271,...,11.160285,157.462679,1463.720435,7.318602,67876,151,492,548,40.194444,23.333333
1,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H...,0,0.0,0.0,1,10,122.306081,2.416770,4.780587,122.306081,...,11.127219,155.307010,1441.772471,7.067512,63926,148,478,531,41.194444,22.944444
2,C1=CC=C(C(=C1)C(=O)C2=CN=C(S2)NC3=CC=C(C=C3)N)Cl,0,0.0,0.0,0,0,28.571924,2.388244,4.711441,28.571924,...,9.874522,69.492153,329.038961,9.677616,1165,31,114,132,6.777778,4.833333
3,C1CCC(=O)C(C1)C2CC(=O)N(C2=O)C3=CC=C(C=C3)Br,0,0.0,0.0,0,0,27.097139,2.488091,4.835581,27.097139,...,10.022470,69.396660,349.031356,9.433280,902,34,112,134,7.138889,4.583333
4,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H...,0,0.0,0.0,1,10,132.321058,2.416813,4.810016,132.321058,...,11.267217,163.855774,1557.759855,7.382748,77098,162,524,585,43.618056,24.527778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3435,CC1=CC(=C(C=C1)S(=O)(=O)N2CCC(CC2)C(=O)NC3=CC=...,1,0.0,0.0,0,0,35.981349,2.454850,4.909699,35.981349,...,10.330160,64.664490,414.197714,7.020300,2578,48,152,178,11.062500,6.250000
3436,C1C(CN1C#N)OCC2=C(C=C(C=C2)OC3=CC=CC=C3)C4=CC=...,1,0.0,0.0,0,0,38.070420,2.424294,4.848587,38.070420,...,10.287218,64.627285,381.147727,7.940578,2372,42,150,175,7.638889,6.583333
3437,C[C@H]1CN(CC2=C1C=CC(=C2)C(=O)NC3=CC(=CC(=C3)C...,1,0.0,0.0,0,2,49.867009,2.452215,4.904315,49.867009,...,10.597809,83.963738,538.266794,7.475928,5435,66,210,245,12.395833,8.277778
3438,CCN1C(=O)C(=CC(=N1)C2=CC=CC(=C2)C3=C(C=CC(=C3)...,1,0.0,0.0,1,0,47.111260,2.439942,4.879884,47.111260,...,10.552709,73.833590,541.071988,9.837673,4596,63,194,229,13.305556,8.138889


In [12]:
zero_mean_columns = num_df_train.columns[num_df_train.mean() == 0]
print(zero_mean_columns)


Index(['ABC', 'ABCGG', 'nB', 'SpAbs_Dt', 'SpMax_Dt', 'SpDiam_Dt', 'SpAD_Dt',
       'SpMAD_Dt', 'LogEE_Dt', 'SM1_Dt',
       ...
       'n5FaHRing', 'n6FaHRing', 'n7FaHRing', 'n11FaHRing', 'n4FARing',
       'n5FARing', 'n6FARing', 'n4FAHRing', 'n5FAHRing', 'n6FAHRing'],
      dtype='object', length=233)


Out of 1615 columns, 233 are columns which have mean as 0

Now, it could also be because it is a balanced dataset that the mean is 0 as we have equal distribution on both sides

In [13]:
unique_values = {}
for column in zero_mean_columns:
    unique_values[column] = num_df_train[column].unique()

# Save the unique values as a variable
unique_values_variable = unique_values

# Filter out columns with only one unique value
filtered_columns = [column for column, values in unique_values.items() if len(values) > 2]

# # Save the filtered columns as a variable
# filtered_columns_variable = filtered_columns


In [14]:
filtered_columns

[]

This is not making sense, let's check out how the things are there

In [15]:
df_train['ABC'].unique()

array([0.])

In [16]:
# Filter the unique_values dictionary to find columns where the only unique value is [0.]
columns_with_only_zero = {column: values for column, values in unique_values.items() if len(values) == 1 and values[0] == 0}

# To get just the names of these columns, you can extract the keys from the above dictionary:
columns_with_only_zero_names = list(columns_with_only_zero.keys())

# Now, columns_with_only_zero_names contains the names of the columns where the only unique value is 0.


In [17]:
columns_with_only_zero_names

['ABC',
 'ABCGG',
 'nB',
 'SpAbs_Dt',
 'SpMax_Dt',
 'SpDiam_Dt',
 'SpAD_Dt',
 'SpMAD_Dt',
 'LogEE_Dt',
 'SM1_Dt',
 'VE1_Dt',
 'VE2_Dt',
 'VE3_Dt',
 'VR1_Dt',
 'VR2_Dt',
 'VR3_Dt',
 'DetourIndex',
 'NsLi',
 'NssBe',
 'NssssBe',
 'NssBH',
 'NsssB',
 'NssssB',
 'NssNH2',
 'NsSiH3',
 'NssSiH2',
 'NsssSiH',
 'NssssSi',
 'NsPH2',
 'NssPH',
 'NsssP',
 'NsssssP',
 'NsSH',
 'NsGeH3',
 'NssGeH2',
 'NsssGeH',
 'NssssGe',
 'NsAsH2',
 'NssAsH',
 'NsssAs',
 'NsssdAs',
 'NsssssAs',
 'NsSeH',
 'NdSe',
 'NssSe',
 'NaaSe',
 'NdssSe',
 'NddssSe',
 'NsSnH3',
 'NssSnH2',
 'NsssSnH',
 'NssssSn',
 'NsPbH3',
 'NssPbH2',
 'NsssPbH',
 'NssssPb',
 'SsLi',
 'SssBe',
 'SssssBe',
 'SssBH',
 'SsssB',
 'SssssB',
 'SssNH2',
 'SsSiH3',
 'SssSiH2',
 'SsssSiH',
 'SssssSi',
 'SsPH2',
 'SssPH',
 'SsssP',
 'SsssssP',
 'SsSH',
 'SsGeH3',
 'SssGeH2',
 'SsssGeH',
 'SssssGe',
 'SsAsH2',
 'SssAsH',
 'SsssAs',
 'SsssdAs',
 'SsssssAs',
 'SsSeH',
 'SdSe',
 'SssSe',
 'SaaSe',
 'SdssSe',
 'SddssSe',
 'SsSnH3',
 'SssSnH2',
 'SsssSnH',

with this, it is confirmed that these 233 columns all have only 0 values through the entire dataset

# Now, we gotta check for the other test_set

In [18]:
df_test = pd.read_csv("C:\Programming\Biotech_Dataset\Test2D.csv")
df_test.head()

,SMILES,Cls,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,CC1=C(C(=NO1)C)CN2C(=O)C(NC2=O)(CC3=CC=CC=C3)C...,0,NaN,NaN,0,0,37.535578,2.537654,4.928643,37.535578,...,10.345767,81.039345,389.173942,7.484114,2134,46,156,186,8.951389,6.319444
1,CN1CC[N+]2=C1SC=C2CN3C(=O)C4=CC=CC=C4C3=O.[I-],0,NaN,NaN,1,1,NaN,NaN,NaN,NaN,...,10.164659,74.001707,426.985146,11.860698,2100000888,35,120,149,NaN,4.472222
2,CC1=CC(=NC(=N1)N2CCOCC2)OC3=CC=CC=C3,0,NaN,NaN,0,0,26.392907,2.360309,4.720619,26.392907,...,9.729432,53.298068,271.132077,7.327894,847,27,102,116,5.055556,4.444444
3,CC(C)CCN1C2=C(N=C1N3CCOCC3)N(C(=O)N(C2=O)CC(=O...,0,NaN,NaN,0,0,34.043250,2.568253,5.023034,34.043250,...,10.322165,76.726891,378.201553,7.135878,1698,46,142,170,10.472222,5.916667
4,CCN(C1CCS(=O)(=O)C1)C(=O)CN2C3=CC=CC=C3OC2=O,0,NaN,NaN,0,0,28.832447,2.461932,4.775535,28.832447,...,10.103608,73.738459,338.093643,8.246186,1189,35,124,147,8.340278,4.972222


In [19]:
df_test['Cls'].unique()

array([0, 1], dtype=int64)

In [20]:
df_test['Cls'].value_counts()

Cls
0    430
1    430
Name: count, dtype: int64

In [21]:
df_test['Cls'].isna().sum()

0

Cool, so the test set is also perfectly balanced (as all things should be)

In [22]:
df_test.isna().sum()

SMILES        0
Cls           0
ABC         860
ABCGG       860
nAcid         0
           ... 
WPol          0
Zagreb1       0
Zagreb2       0
mZagreb1     11
mZagreb2      0
Length: 1615, dtype: int64

In [23]:
df_test.fillna(0, inplace=True)

In [24]:
df_test.describe()

,Cls,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
count,860.000000,860.0,860.0,860.000000,860.000000,860.000000,860.000000,860.000000,860.000000,860.000000,...,860.000000,860.000000,860.000000,860.000000,8.600000e+02,860.000000,860.000000,860.000000,860.000000,860.000000
mean,0.500000,0.0,0.0,0.082558,0.279070,37.253180,2.396482,4.743424,37.253180,1.259698,...,10.208453,74.561277,419.020479,8.155511,9.023572e+07,45.773256,154.158140,180.646512,9.560505,6.467377
std,0.500291,0.0,0.0,0.349852,0.609417,12.856632,0.327773,0.646497,12.856632,0.171851,...,0.409827,14.132181,126.822173,1.067345,9.706506e+08,16.986428,51.355123,61.973468,3.195101,1.928190
min,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,8.566174,42.927994,151.045570,5.769230,1.140000e+02,10.000000,44.000000,48.000000,0.000000,2.444444
25%,0.000000,0.0,0.0,0.000000,0.000000,28.750826,2.399570,4.745137,28.750826,1.258693,...,9.947002,64.930697,322.848933,7.353421,1.241000e+03,33.000000,116.000000,136.000000,7.388889,5.000000
50%,0.500000,0.0,0.0,0.000000,0.000000,35.850699,2.438571,4.841475,35.850699,1.284706,...,10.226766,72.619912,400.625008,8.003036,2.223000e+03,43.000000,146.000000,171.000000,9.225694,6.138889
75%,1.000000,0.0,0.0,0.000000,0.000000,44.808520,2.480285,4.914342,44.808520,1.308119,...,10.492246,81.915296,487.167615,8.839025,4.057000e+03,57.000000,184.000000,217.000000,11.395833,7.638889
max,1.000000,0.0,0.0,2.000000,3.000000,83.613491,2.911639,5.295820,83.613491,1.399352,...,11.185990,122.162138,923.536806,11.999503,1.960000e+10,125.000000,350.000000,420.000000,25.069444,14.569444


In [25]:
num_df_test = df_test.drop('SMILES', axis=1)
num_df_test.head()

,Cls,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0,0.0,0.0,0,0,37.535578,2.537654,4.928643,37.535578,1.294330,...,10.345767,81.039345,389.173942,7.484114,2134,46,156,186,8.951389,6.319444
1,0,0.0,0.0,1,1,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.164659,74.001707,426.985146,11.860698,2100000888,35,120,149,0.000000,4.472222
2,0,0.0,0.0,0,0,26.392907,2.360309,4.720619,26.392907,1.319645,...,9.729432,53.298068,271.132077,7.327894,847,27,102,116,5.055556,4.444444
3,0,0.0,0.0,0,0,34.043250,2.568253,5.023034,34.043250,1.260861,...,10.322165,76.726891,378.201553,7.135878,1698,46,142,170,10.472222,5.916667
4,0,0.0,0.0,0,0,28.832447,2.461932,4.775535,28.832447,1.253585,...,10.103608,73.738459,338.093643,8.246186,1189,35,124,147,8.340278,4.972222


In [26]:
num_df_test

,Cls,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0,0.0,0.0,0,0,37.535578,2.537654,4.928643,37.535578,1.294330,...,10.345767,81.039345,389.173942,7.484114,2134,46,156,186,8.951389,6.319444
1,0,0.0,0.0,1,1,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.164659,74.001707,426.985146,11.860698,2100000888,35,120,149,0.000000,4.472222
2,0,0.0,0.0,0,0,26.392907,2.360309,4.720619,26.392907,1.319645,...,9.729432,53.298068,271.132077,7.327894,847,27,102,116,5.055556,4.444444
3,0,0.0,0.0,0,0,34.043250,2.568253,5.023034,34.043250,1.260861,...,10.322165,76.726891,378.201553,7.135878,1698,46,142,170,10.472222,5.916667
4,0,0.0,0.0,0,0,28.832447,2.461932,4.775535,28.832447,1.253585,...,10.103608,73.738459,338.093643,8.246186,1189,35,124,147,8.340278,4.972222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,1,0.0,0.0,0,0,56.456450,2.476213,4.919933,56.456450,1.283101,...,10.833070,98.076914,613.256816,7.665710,7590,75,244,292,14.590278,9.250000
856,1,0.0,0.0,0,1,58.618278,2.442469,4.884938,58.618278,1.274310,...,10.668700,83.626744,665.192023,8.420152,8645,75,234,272,16.777778,10.305556
857,1,0.0,0.0,0,0,53.924883,2.421571,4.839234,53.924883,1.283926,...,10.621644,100.225933,623.128914,8.776464,6238,65,222,260,13.666667,9.166667
858,1,0.0,0.0,0,1,67.281076,2.485928,4.931621,67.281076,1.293867,...,10.942827,106.750343,706.406735,6.925556,12390,86,284,337,15.701389,11.083333


In [27]:
zero_mean_columns_test = num_df_test.columns[num_df_test.mean() == 0]
print(zero_mean_columns_test)

Index(['ABC', 'ABCGG', 'nB', 'SpAbs_Dt', 'SpMax_Dt', 'SpDiam_Dt', 'SpAD_Dt',
       'SpMAD_Dt', 'LogEE_Dt', 'SM1_Dt',
       ...
       'n5FaHRing', 'n6FaHRing', 'n7FaHRing', 'n11FaHRing', 'n12FaHRing',
       'n4FARing', 'n5FARing', 'n4FAHRing', 'n5FAHRing', 'n7FAHRing'],
      dtype='object', length=261)


huh, here there are 261 columns with mean as 0

In [28]:
unique_values_test = {}
for column in zero_mean_columns_test:
    unique_values_test[column] = num_df_test[column].unique()

# Save the unique values as a variable
# unique_values_variable = unique_values

# Filter out columns with only one unique value
filtered_columns = [column for column,
                    values in unique_values_test.items() if len(values) > 2]

# # Save the filtered columns as a variable
# filtered_columns_variable = filtered_columns

In [29]:
# Filter the unique_values dictionary to find columns where the only unique value is [0.]
columns_with_only_zero_test = {column: values for column, values in unique_values_test.items(
) if len(values) == 1 and values[0] == 0}

# To get just the names of these columns, you can extract the keys from the above dictionary:
columns_with_only_zero_names_test = list(columns_with_only_zero_test.keys())

# Now, columns_with_only_zero_names contains the names of the columns where the only unique value is 0.

In [30]:
columns_with_only_zero_names_test

['ABC',
 'ABCGG',
 'nB',
 'SpAbs_Dt',
 'SpMax_Dt',
 'SpDiam_Dt',
 'SpAD_Dt',
 'SpMAD_Dt',
 'LogEE_Dt',
 'SM1_Dt',
 'VE1_Dt',
 'VE2_Dt',
 'VE3_Dt',
 'VR1_Dt',
 'VR2_Dt',
 'VR3_Dt',
 'DetourIndex',
 'NsLi',
 'NssBe',
 'NssssBe',
 'NssBH',
 'NsssB',
 'NssssB',
 'NtCH',
 'NddC',
 'NsNH3',
 'NssNH2',
 'NsssNH',
 'NssssN',
 'NsSiH3',
 'NssSiH2',
 'NsssSiH',
 'NssssSi',
 'NsPH2',
 'NssPH',
 'NsssP',
 'NsssssP',
 'NsSH',
 'NdssS',
 'NsGeH3',
 'NssGeH2',
 'NsssGeH',
 'NssssGe',
 'NsAsH2',
 'NssAsH',
 'NsssAs',
 'NsssdAs',
 'NsssssAs',
 'NsSeH',
 'NdSe',
 'NssSe',
 'NaaSe',
 'NdssSe',
 'NddssSe',
 'NsSnH3',
 'NssSnH2',
 'NsssSnH',
 'NssssSn',
 'NsI',
 'NsPbH3',
 'NssPbH2',
 'NsssPbH',
 'NssssPb',
 'SsLi',
 'SssBe',
 'SssssBe',
 'SssBH',
 'SsssB',
 'SssssB',
 'StCH',
 'SddC',
 'SsNH3',
 'SssNH2',
 'SsssNH',
 'SssssN',
 'SsSiH3',
 'SssSiH2',
 'SsssSiH',
 'SssssSi',
 'SsPH2',
 'SssPH',
 'SsssP',
 'SsssssP',
 'SsSH',
 'SdssS',
 'SsGeH3',
 'SssGeH2',
 'SsssGeH',
 'SssssGe',
 'SsAsH2',
 'SssAsH',
 'Ss

So here, 261 columns have all 0 values. This is interesting

In [31]:
# finding common columns from columns_with_only_zero_names and columns_with_only_zero_names_test
common_columns = list(set(columns_with_only_zero_names) & set(columns_with_only_zero_names_test))

common_columns

['MINssBH',
 'VE3_Dt',
 'nG12aRing',
 'NddssSe',
 'MINssssGe',
 'NssssGe',
 'SsAsH2',
 'MAXsssSnH',
 'MINssPH',
 'SsssB',
 'NssBH',
 'n5FARing',
 'NsssP',
 'SpMax_Dt',
 'NdSe',
 'n3aRing',
 'n4FaHRing',
 'MINsssssAs',
 'NsGeH3',
 'SddssSe',
 'n8aRing',
 'NssssSi',
 'MINssSiH2',
 'MINsSnH3',
 'SsSeH',
 'MINssssPb',
 'SsssPbH',
 'MAXssssBe',
 'MINsSH',
 'SssPbH2',
 'NsssPbH',
 'MAXsssGeH',
 'MINssGeH2',
 'SssssB',
 'NsSiH3',
 'nB',
 'n9Ring',
 'MAXsSiH3',
 'n11FaHRing',
 'MAXsssP',
 'MINssSe',
 'n12ARing',
 'MAXsSH',
 'n11Ring',
 'NssSiH2',
 'MAXsPH2',
 'MAXssSnH2',
 'MINsLi',
 'MAXsssB',
 'NsssSiH',
 'MINsPH2',
 'SssGeH2',
 'SssSe',
 'NaaSe',
 'MINsssPbH',
 'n4FARing',
 'NssssBe',
 'NssssB',
 'MAXssPbH2',
 'SsssAs',
 'n11FaRing',
 'SsssssP',
 'MINsSeH',
 'n9aHRing',
 'MINssNH2',
 'MINddssSe',
 'NssPH',
 'SsssP',
 'SssssBe',
 'MAXssSe',
 'n3aHRing',
 'VE2_Dt',
 'MINsGeH3',
 'NsssB',
 'SssssSn',
 'MAXsssAs',
 'NsLi',
 'MINssBe',
 'SsssSnH',
 'SssssPb',
 'MAXsssSiH',
 'MAXsssdAs',
 'MINsss

# 227 values are 0 in both the columns

Let's drop these columns. Important note is to drop these columns from the custom input one as well

In [32]:
# drop colunmns with reference as common_columns
num_df_train = num_df_train.drop(common_columns, axis=1)
num_df_test = num_df_test.drop(common_columns, axis=1)

In [33]:
num_df_train

,Cls,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0,1,10,126.291271,2.416707,4.779198,126.291271,1.226129,5.510144,5.258896,...,11.160285,157.462679,1463.720435,7.318602,67876,151,492,548,40.194444,23.333333
1,0,1,10,122.306081,2.416770,4.780587,122.306081,1.210951,5.485277,5.306958,...,11.127219,155.307010,1441.772471,7.067512,63926,148,478,531,41.194444,22.944444
2,0,0,0,28.571924,2.388244,4.711441,28.571924,1.298724,4.022190,4.121727,...,9.874522,69.492153,329.038961,9.677616,1165,31,114,132,6.777778,4.833333
3,0,0,0,27.097139,2.488091,4.835581,27.097139,1.290340,3.985187,3.966880,...,10.022470,69.396660,349.031356,9.433280,902,34,112,134,7.138889,4.583333
4,0,1,10,132.321058,2.416813,4.810016,132.321058,1.213955,5.567384,5.479180,...,11.267217,163.855774,1557.759855,7.382748,77098,162,524,585,43.618056,24.527778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3435,1,0,0,35.981349,2.454850,4.909699,35.981349,1.240736,4.287898,4.177238,...,10.330160,64.664490,414.197714,7.020300,2578,48,152,178,11.062500,6.250000
3436,1,0,0,38.070420,2.424294,4.848587,38.070420,1.312773,4.302425,4.531875,...,10.287218,64.627285,381.147727,7.940578,2372,42,150,175,7.638889,6.583333
3437,1,0,2,49.867009,2.452215,4.904315,49.867009,1.278641,4.597720,4.885288,...,10.597809,83.963738,538.266794,7.475928,5435,66,210,245,12.395833,8.277778
3438,1,1,0,47.111260,2.439942,4.879884,47.111260,1.273277,4.532706,5.120482,...,10.552709,73.833590,541.071988,9.837673,4596,63,194,229,13.305556,8.138889


In [34]:
num_df_test

,Cls,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0,0,0,37.535578,2.537654,4.928643,37.535578,1.294330,4.308329,4.266030,...,10.345767,81.039345,389.173942,7.484114,2134,46,156,186,8.951389,6.319444
1,0,1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.164659,74.001707,426.985146,11.860698,2100000888,35,120,149,0.000000,4.472222
2,0,0,0,26.392907,2.360309,4.720619,26.392907,1.319645,3.926572,3.991727,...,9.729432,53.298068,271.132077,7.327894,847,27,102,116,5.055556,4.444444
3,0,0,0,34.043250,2.568253,5.023034,34.043250,1.260861,4.221651,4.163356,...,10.322165,76.726891,378.201553,7.135878,1698,46,142,170,10.472222,5.916667
4,0,0,0,28.832447,2.461932,4.775535,28.832447,1.253585,4.075477,4.244018,...,10.103608,73.738459,338.093643,8.246186,1189,35,124,147,8.340278,4.972222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,1,0,0,56.456450,2.476213,4.919933,56.456450,1.283101,4.729835,6.041026,...,10.833070,98.076914,613.256816,7.665710,7590,75,244,292,14.590278,9.250000
856,1,0,1,58.618278,2.442469,4.884938,58.618278,1.274310,4.735867,5.394485,...,10.668700,83.626744,665.192023,8.420152,8645,75,234,272,16.777778,10.305556
857,1,0,0,53.924883,2.421571,4.839234,53.924883,1.283926,4.690113,5.644491,...,10.621644,100.225933,623.128914,8.776464,6238,65,222,260,13.666667,9.166667
858,1,0,1,67.281076,2.485928,4.931621,67.281076,1.293867,4.892225,5.839208,...,10.942827,106.750343,706.406735,6.925556,12390,86,284,337,15.701389,11.083333


# Not gonna do any further preprocessing

(for now)

In [36]:
# converting common_columns to a series and converting it to a csv file
common_columns = pd.Series(common_columns)
common_columns.to_csv('common_columns.csv', index=False)

In [37]:
# converting num_df_train and num_df_test to csv files
num_df_train.to_csv('num_df_train.csv', index=False)
num_df_test.to_csv('num_df_test.csv', index=False)